In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
from tensorflow import math as tfm
#import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *
import h5py
from sklearn.preprocessing import OneHotEncoder
from cbrain.climate_invariant import *
import yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Load coordinates (just pick any file from the climate model run)
coor = xr.open_dataset("/DFS-L/DATA/pritchard/ankitesg/data/CESM2_f19_v13_updated_NN_pelayout01_ens_07.cam.h1.2003-01-22-00000.nc",\
                    decode_times=False)
lat = coor.lat; lon = coor.lon; lev = coor.lev;
DATA_DIR = '/DFS-L/DATA/pritchard/ankitesg/datav3/'
hyam = coor.hyam
hybm = coor.hybm
scale_dict = load_pickle('/export/nfs0home/ankitesg/tom/CBRAIN-CAM/nn_config/scale_dicts/2020_10_16_scale_dict_RG.pkl')['scale_dict_RG']

In [3]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        self.new_idx = np.concatenate((self.input_idxs[8:26],self.input_idxs[34:52],self.input_idxs[60:78],self.input_idxs[86:104],\
        self.input_idxs[104:]))
        self.new_idx = np.concatenate((np.arange(8,26),np.arange(34,52),np.arange(60,78),np.arange(86,104),np.arange(104,108)))


    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
#         print(self.new_idx)
        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        return X_norm[:,self.new_idx], Y

In [4]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [5]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [6]:
train_gen_bf = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [7]:
valid_gen_bf = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [8]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(320, activation='relu'))
for i in range(10):
    model.add(Dense(320, activation='relu'))
model.add(Dense(112, activation='linear'))

In [16]:
model.load_weights('/export/nfs0home/ankitesg/CBRAIN-CAM/notebooks/ankitesh-devlog/random_search/RGBF_TrimV2/trial_1db6530249e0ce7940f6617df2a27a6d/checkpoints/epoch_0/checkpoint')

In [17]:
opt = tf.keras.optimizers.Adam()

In [18]:
model.compile(optimizer=opt, loss='mse')

In [19]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'BF_RG_TrimTuned_V1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [20]:
with tf.device('/gpu:1'):
    Nep = 10
    model.fit_generator(train_gen_bf, epochs=Nep, validation_data=valid_gen_bf
                        ,callbacks=[earlyStopping, mcp_save])

Epoch 1/10
42120/42120 [==============================] - 1218s 29ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 164.8028 - val_loss: 171.7823
Epoch 2/10
42120/42120 [==============================] - 1309s 31ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 164.0902 - val_loss: 171.1855
Epoch 3/10
42120/42120 [==============================] - 1280s 30ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 163.4447 - val_loss: 167.2046
Epoch 4/10
42120/42120 [==============================] - 1303s 31ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 162.9060 - val_loss: 170.2918
Epoch 5/10
42120/42120 [==============================] - 1308s 31ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 162.4249 - val_loss: 165.7569
Epoch 6/10
42120/42120 [==============================] - 1346s 32ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 161.9562 - val_loss: 166.9073
Epoch 7/10
42120/42120 [==============================] - 1305s 31ms/step - batch: 21059.5000 - size

## RH Fine tuned

In [9]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        self.new_idx = np.concatenate((np.arange(8,26),np.arange(34,52),np.arange(60,78),np.arange(86,104),np.arange(104,108)))
        self.new_output_idx = np.concatenate((np.arange(8,26),np.arange(26,52),np.arange(60,78),np.arange(86,104),np.arange(104,112)))

        if self.rh_trans:
            self.qv2rhLayer = QV2RHNumpyReal(self.inp_sub,self.inp_div,inp_subRH,inp_divRH,hyam,hybm)

    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size
        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        X_result = X_norm
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result)
        return X_result[:,self.new_idx], Y

In [10]:
in_vars_RH = ['RH','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars_RH =['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

TRAINFILE_RH = 'RG_RH_M4K_NORM_train_shuffle.nc'
NORMFILE_RH = 'RG_RH_M4K_NORM_norm.nc'
VALIDFILE_RH = 'RG_RH_M4K_NORM_valid.nc'
BASE_DIR = '/DFS-L/DATA/pritchard/ankitesg/'

In [11]:
train_gen_RH = DataGenerator(
    data_fn = f"{BASE_DIR}datav4/{TRAINFILE_RH}",
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = f"{BASE_DIR}datav4/{NORMFILE_RH}",
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True
)

In [12]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [13]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [14]:
train_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [15]:
valid_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [16]:
train_gen[0][0]

array([[ 0.16899723,  0.05792288,  0.00289376, ..., -0.23110868,
         0.00347698,  0.01947967],
       [ 0.06702778,  0.03369261,  0.07741939, ..., -0.19929254,
        -0.03714576, -0.0280101 ],
       [-0.01433151, -0.01879989, -0.01172942, ..., -0.23110868,
        -0.04334731, -0.04169009],
       ...,
       [-0.06347687, -0.06646132, -0.06408557, ..., -0.08484679,
        -0.04811756, -0.04203105],
       [-0.05961715, -0.06118552, -0.05785822, ..., -0.23110868,
        -0.01094688, -0.03889326],
       [-0.06071537, -0.06367046, -0.05897022, ..., -0.23110868,
         0.00926478, -0.02465681]], dtype=float32)

In [17]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(320, activation='relu'))
for i in range(10):
    model.add(Dense(320, activation='relu'))
model.add(Dense(112, activation='linear'))

In [18]:
opt = tf.keras.optimizers.Adam()

In [19]:
model.compile(optimizer=opt, loss='mse')

In [20]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'RH_RG_TrimTuned_V1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [21]:
with tf.device('/gpu:1'):
    Nep = 12
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen
                        ,callbacks=[earlyStopping, mcp_save])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/12
42116/42120 [============================>.] - ETA: 0s - batch: 21057.5000 - size: 1024.0000 - loss: 232.2645WARNING:tensorflow:From /export/home/ankitesg/anaconda3/envs/CBrainEnv2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
42120/42120 [==============================] - 1772s 42ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 232.2593 - val_loss: 193.7016
Epoch 2/12
42120/42120 [==============================] - 2050s 49ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 186.7367 - val_loss: 185.6101
Epoch 3/12
42120/42120 [==============================] - 1994s 47ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 179.

### Bounded Relu

In [22]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(320))
model.add(ReLU(max_value=100))
for i in range(10):
    model.add(Dense(320))
    model.add(ReLU(max_value=100))
model.add(Dense(112, activation='linear'))

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 320)               24640     
_________________________________________________________________
re_lu (ReLU)                 (None, 320)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 320)               102720    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 320)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 320)               102720    
_________________________________________________________________
re_lu_2 (ReLU)               (None, 320)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 320)              

In [24]:
opt = tf.keras.optimizers.Adam()

In [25]:
model.compile(optimizer=opt, loss='mse')

In [26]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'RH_RG_TrimTuned_Bounded_V2.h5',save_best_only=True, monitor='val_loss', mode='min')

In [27]:
with tf.device('/gpu:1'):
    Nep = 12
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen
                        ,callbacks=[earlyStopping, mcp_save])

Epoch 1/12
42120/42120 [==============================] - 2040s 48ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 271.5182 - val_loss: 228.8170
Epoch 2/12
42120/42120 [==============================] - 2122s 50ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 218.1427 - val_loss: 213.9692
Epoch 3/12
42120/42120 [==============================] - 1996s 47ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 208.7946 - val_loss: 210.6959
Epoch 4/12
42120/42120 [==============================] - 2050s 49ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 201.2733 - val_loss: 205.9705
Epoch 5/12
42120/42120 [==============================] - 1933s 46ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 195.8432 - val_loss: 201.9331
Epoch 6/12
42120/42120 [==============================] - 1982s 47ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 191.7715 - val_loss: 193.0540
Epoch 7/12
27447/42120 [==================>...........] - ETA: 10:50 - batch: 13723.0000 - size: 102

KeyboardInterrupt: 

## Only Liq and Ice Output

In [32]:
in_vars_RH = ['RH','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['CLDLIQBCTEND', 'CLDICEBCTEND']

TRAINFILE_RH = 'RG_RH_M4K_NORM_train_shuffle.nc'
NORMFILE_RH = 'RG_RH_M4K_NORM_norm.nc'
VALIDFILE_RH = 'RG_RH_M4K_NORM_valid.nc'
BASE_DIR = '/DFS-L/DATA/pritchard/ankitesg/'

In [33]:
train_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [34]:
valid_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [35]:
valid_gen[0][1].shape

(1024, 52)

In [36]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(320, activation='relu'))
for i in range(10):
    model.add(Dense(320, activation='relu'))
model.add(Dense(52, activation='linear'))

In [37]:
opt = tf.keras.optimizers.Adam()

In [38]:
model.compile(optimizer=opt, loss='mse')

In [39]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'RH_RG_TrimTuned_PartialOP_V1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [40]:
with tf.device('/gpu:1'):
    Nep = 15
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen
                        ,callbacks=[earlyStopping, mcp_save])

Epoch 1/15
42120/42120 [==============================] - 1314s 31ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 16.6594 - val_loss: 15.8567
Epoch 2/15
42120/42120 [==============================] - 2047s 49ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 15.4489 - val_loss: 15.2709
Epoch 3/15
42120/42120 [==============================] - 1817s 43ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 15.0161 - val_loss: 14.9815
Epoch 4/15
42120/42120 [==============================] - 1870s 44ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 14.7107 - val_loss: 14.6259
Epoch 5/15
42120/42120 [==============================] - 927s 22ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 14.5298 - val_loss: 14.6514
Epoch 6/15
42120/42120 [==============================] - 884s 21ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 14.4154 - val_loss: 14.6659
Epoch 7/15
42120/42120 [==============================] - 918s 22ms/step - batch: 21059.5000 - size: 1024.0000 - l